# Generate Reports w/ GPT

In [1]:
import pandas as pd
import json
import asyncio

from kruppe.prompts.experiments import VANILLA_QA_SYSTEM
from kruppe.llm import OpenAILLM

async def vanilla_llm_response(llm: OpenAILLM, question: str, n: int = 1, service_tier="auto"):
    
    messages = [
        {"role": "system", "content": VANILLA_QA_SYSTEM},
        {"role": "user", "content": question},
    ]

    async with asyncio.TaskGroup() as tg:
        tasks = []
        for i in range(n):
            if llm.model == "gpt-4o-search-preview":
                coro = llm.async_generate(messages, web_search_options=
                    {
                        "user_location": 
                        {
                            "type": "approximate",
                            "approximate": {
                                "country": "US",
                            }
                        },
                        "search_context_size": "medium",
                    },
                    service_tier=service_tier,)
            else:
                coro = llm.async_generate(messages)
            task = tg.create_task(coro)
            tasks.append(task)
    responses = [task.result() for task in tasks]
    return responses

In [2]:
df = pd.read_csv("./reports.csv", index_col=False)
df

,category,human_report_loc,question
0,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,How is ConocoPhillips positioned to deliver su...
1,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the outlook for Chevron Corporation’s ...
2,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What are the updated financial prospects and i...
3,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the investment outlook for Exxon Mobil...
4,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the investment outlook for ConocoPhill...
...,...,...,...
68,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,"What are the key expectations, product announc..."
69,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,"What are the current trends, risks, and invest..."
70,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,What is the current and projected financial an...
71,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,"What are the current trends, challenges, and o..."


## GPT 4o with web search

this is EXPENSIVE

In [14]:
llm = OpenAILLM(model="gpt-4o-search-preview")
rs = await vanilla_llm_response(llm, "NVIDIA's performance recently due to Trump's tariffs", n=2)

for r in rs:
    print(r.text)

NVIDIA Corporation, a leading designer of graphics processing units (GPUs) and artificial intelligence (AI) hardware, has recently experienced notable fluctuations in its stock performance, largely influenced by the trade policies implemented under President Donald Trump's administration.

**Immediate Market Reaction**

In early 2025, the Trump administration announced a series of tariffs targeting imports from key trading partners, including China, Mexico, and Canada. These tariffs, particularly a 10% levy on Chinese goods and a 25% tariff on imports from Mexico and Canada, prompted immediate concerns among investors about potential disruptions in global supply chains and increased production costs for technology companies like NVIDIA. Consequently, NVIDIA's stock price experienced a significant decline. On March 3, 2025, the company's shares fell by approximately 8.7%, closing at their lowest level since mid-September of the previous year. This decline was part of a broader market se

In [19]:

llm = OpenAILLM(model="gpt-4o-search-preview")

async def task(llm, question, n, i):
    responses = await vanilla_llm_response(llm, question, n)

    result_dict = {
        "research_reports": [response.model_dump(mode='json') for response in responses],
    }

    with open(f"./vanilla_report_4o_search/report_{i}.json", "w") as f:
        json.dump(result_dict, f, indent=4)
    
    print(f"Finished report {i} with {n} reports")
    
    return responses

async with asyncio.TaskGroup() as tg:
    for i, row in df.iterrows():
        question = row['question']
        tg.create_task(task(llm, question, 1, i))


Finished report 61 with 6 reports
Finished report 0 with 6 reports
Finished report 51 with 6 reports
Finished report 12 with 6 reports
Finished report 63 with 6 reports
Finished report 53 with 6 reports
Finished report 23 with 6 reports
Finished report 50 with 6 reports
Finished report 70 with 6 reports
Finished report 4 with 6 reports
Finished report 41 with 6 reports
Finished report 6 with 6 reports
Finished report 47 with 6 reports
Finished report 20 with 6 reports
Finished report 62 with 6 reports
Finished report 49 with 6 reports
Finished report 42 with 6 reports
Finished report 9 with 6 reports
Finished report 17 with 6 reports
Finished report 8 with 6 reports
Finished report 5 with 6 reports
Finished report 15 with 6 reports
Finished report 19 with 6 reports
Finished report 39 with 6 reports
Finished report 55 with 6 reports
Finished report 25 with 6 reports
Finished report 32 with 6 reports
Finished report 36 with 6 reports
Finished report 67 with 6 reports
Finished report 28 w

## GPT 4.1

In [16]:
llm = OpenAILLM(model="gpt-4.1")
rs = await vanilla_llm_response(llm, "NVIDIA's performance recently due to Trump's tariffs", n=2)

for r in rs:
    print(r.text)

**Title:**  
The Impact of Trump's Tariffs on NVIDIA's Recent Performance

**Thesis:**  
NVIDIA, as a leading semiconductor company, has experienced nuanced but significant consequences from US trade policy shifts—specifically, tariffs instituted during the Trump administration. These tariffs, primarily aimed at China, have impacted NVIDIA’s supply chain costs, sales strategies, and global market positioning, but the company’s innovative product cycle and diversification have mitigated some negative effects, resulting in continued strong performance in the face of trade headwinds.

---

**1. Context and Implementation of Trump-Era Tariffs**

In 2018-2019, the Trump administration imposed tariffs on hundreds of billions of dollars of Chinese imports under Section 301 of the Trade Act. The scope included semiconductors and related electronics, directly affecting NVIDIA, which relies on a global supply chain—with many components or assembly processes tied to China.

**2. Direct Effects on

In [9]:

llm = OpenAILLM(model="gpt-4.1")
llm.async_client.timeout = 900
llm.sync_client.timeout = 900

async def task(llm, question, n, i):
    responses = await vanilla_llm_response(llm, question, n, service_tier="flex")

    result_dict = {
        "research_reports": [response.model_dump(mode='json') for response in responses],
    }

    with open(f"./vanilla_report_4.1/report_{i}.json", "w") as f:
        json.dump(result_dict, f, indent=4)
    
    print(f"Finished report {i} with {num_reports} reports")
    
    return responses


tasks = []
for i, row in df.iterrows():
    with open(f"./kruppe_report/report_{i}.json", "r") as f:
        data = json.load(f)
        num_reports = len(data['research_reports'])
        question = row['question']
    
    running_task = asyncio.create_task(task(llm, question, num_reports, i))
    tasks.append(running_task)

await asyncio.gather(*tasks)


Finished report 43 with 6 reports
Finished report 42 with 6 reports
Finished report 63 with 6 reports
Finished report 57 with 6 reports
Finished report 68 with 6 reports
Finished report 50 with 6 reports
Finished report 55 with 6 reports
Finished report 66 with 6 reports
Finished report 67 with 6 reports
Finished report 64 with 6 reports
Finished report 23 with 6 reports
Finished report 62 with 6 reports
Finished report 6 with 6 reports
Finished report 37 with 6 reports
Finished report 27 with 6 reports
Finished report 18 with 6 reports
Finished report 44 with 6 reports
Finished report 56 with 6 reports
Finished report 49 with 6 reports
Finished report 53 with 6 reports
Finished report 65 with 6 reports
Finished report 31 with 6 reports
Finished report 25 with 6 reports
Finished report 51 with 6 reports
Finished report 13 with 6 reports
Finished report 61 with 6 reports
Finished report 32 with 6 reports
Finished report 45 with 6 reports
Finished report 35 with 6 reports
Finished report

[[Response(text="**ConocoPhillips Positioning for Sustained Free Cash Flow Growth and Shareholder Returns: An Analysis (2024)**\n\n**Thesis:**  \nConocoPhillips is strategically positioned to deliver sustained free cash flow (FCF) growth and robust shareholder returns over the coming years. The company's low-cost asset base, disciplined capital allocation, strong balance sheet, and shareholder-focused distribution strategy underpin its favorable outlook, even amid the cyclical volatility of oil markets and an evolving energy landscape.\n\n---\n\n**1. Low-Cost, High-Quality Asset Portfolio as a Foundation for FCF**\n\nConocoPhillips has consistently focused on building a diversified portfolio of low break-even assets:\n\n- **Resource Quality:** The company’s holdings in the Permian Basin, Eagle Ford, Montney, and LNG projects deliver consistent, low-cost production. As of 2024, management reports an average WTI break-even price below $40/barrel—significantly lower than peers—providing r

# o3 time

In [5]:
llm = OpenAILLM(model="o3")
rs = await vanilla_llm_response(llm, "What is the investment outlook and target valuation for ConocoPhillips, considering its recent financial performance, operational execution, and market risks?", n=1)

print(rs[0].text)
print(llm.price())

INVESTMENT RESEARCH REPORT  
Issuer: ConocoPhillips (NYSE: COP)  
Date: June 2024

CENTRAL THESIS  
ConocoPhillips remains one of the most attractive “pure-play” global exploration & production (E&P) names, distinguished by (1) a structurally low cash-flow breakeven, (2) disciplined capital allocation that hard-wires shareholder returns, and (3) an inventory of high-return projects that can grow volumes 3-4 % CAGR without meaningfully raising its cost base.  At a mid-cycle oil price of US$75 WTI, our discounted cash-flow (DCF) and relative-multiple work point to an equity value of US$145–150 per share—~18 % upside from the recent US$122 close—while still offering a 7–8 % annualized cash-return yield (dividends + buybacks).  We therefore rate the shares Overweight/Outperform, albeit with the customary warning that commodity-price volatility could shift fair value by ±30 %.

SUPPORTING ARGUMENTS  

1. Recent Financial Performance  
• FY-2023 adjusted earnings: US$13.5 bn; operating cash 

In [4]:
llm = OpenAILLM(model="o3")
llm.async_client.timeout = 900
llm.sync_client.timeout = 900
rs = await vanilla_llm_response(llm, "NVIDIA's performance recently due to Trump's tariffs", n=1, service_tier="flex")

print(rs[0].text)
print(llm.price())

Report: Trump-Era Tariffs and Their Actual Influence on NVIDIA’s Recent Performance  

Thesis  
While the 2018–2020 U.S.–China tariffs introduced by the Trump administration did create incremental cost headwinds for graphic-card makers doing final assembly in China, NVIDIA largely neutralised those pressures through temporary tariff exclusions, supply-chain re‐routing and selective price pass-through. The company’s revenue trajectory, margins and share-price appreciation in the same period were dictated far more by secular demand swings (gaming, cryptocurrency, hyperscale AI) and strategic moves (data-centre pivot, software stack expansion) than by the tariff regime per se.

1. Tariffs: What Actually Applied to NVIDIA  
• Section 301 Lists 1-3 (Sep 2018 → May 2019) placed a 10 %→25 % duty on Chinese-origin “printed circuit assemblies for graphics processing units”—the category in which add-in boards (AIBs) for GeForce and Quadro products sit.  
• September 2019 – August 2020: USTR gran

In [8]:
llm = OpenAILLM(model="o3")
llm.async_client.timeout = 900
llm.sync_client.timeout = 900

async def task(llm, question, n, i):
    if llm.price() > 200:
        print(f"Skipping report {i} due to high cost")
        return
    
    responses = await vanilla_llm_response(llm, question, n)

    result_dict = {
        "research_reports": [response.model_dump(mode='json') for response in responses],
    }

    with open(f"./vanilla_report_o3/report_{i}.json", "w") as f:
        json.dump(result_dict, f, indent=4)
    
    print(f"Finished report {i} with {n} reports")
    
    return responses


tasks = []
for i, row in df.iterrows():
    question = row['question']
    
    running_task = asyncio.create_task(task(llm, question, 1, i))
    tasks.append(running_task)

await asyncio.gather(*tasks)


Finished report 5 with 1 reports
Finished report 11 with 1 reports
Finished report 45 with 1 reports
Finished report 43 with 1 reports
Finished report 41 with 1 reports
Finished report 35 with 1 reports
Finished report 7 with 1 reports
Finished report 66 with 1 reports
Finished report 56 with 1 reports
Finished report 62 with 1 reports
Finished report 33 with 1 reports
Finished report 17 with 1 reports
Finished report 13 with 1 reports
Finished report 58 with 1 reports
Finished report 6 with 1 reports
Finished report 51 with 1 reports
Finished report 52 with 1 reports
Finished report 53 with 1 reports
Finished report 12 with 1 reports
Finished report 38 with 1 reports
Finished report 9 with 1 reports
Finished report 47 with 1 reports
Finished report 39 with 1 reports
Finished report 68 with 1 reports
Finished report 60 with 1 reports
Finished report 4 with 1 reports
Finished report 31 with 1 reports
Finished report 46 with 1 reports
Finished report 29 with 1 reports
Finished report 61 

[[Response(text='ConocoPhillips  \nPositioning for Sustained Free-Cash-Flow Growth and Robust Shareholder Returns  \n--------------------------------------------------------------------------  \n\nCentral thesis  \nOver the next five to seven years ConocoPhillips (COP) is set to translate an unusually deep, low-cost resource base and a rigorously tiered capital‐allocation framework into growing free cash flow (FCF) and top-quartile cash yields to shareholders—even as it funds one of the industry’s largest organic investment programs. Three structural advantages underpin this view:  \n\n1. A portfolio that is both low on the global cost curve (≈70 % of resources break even < $40 Brent) and unusually diversified across shale, LNG and conventional oil.  \n2. A reinvestment model that caps sustaining capital at ≈40–50 % of operating cash flow (CFO) while keeping 20-30 % of total capex “price-elastic” (short-cycle U.S. shale) and therefore easily throttled if prices retrench.  \n3. A balanc